# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import joblib

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0



В наличии датасет с расходами услуг мобильной связи (по архивным тарифам) клиентов перешедших на новые тарифы. 
Всего 3214 строк, 5 колонок. 

## Составим корреляционную матрицу признаков 

In [4]:
print(df.corr())

             calls   minutes  messages   mb_used  is_ultra
calls     1.000000  0.982083  0.177385  0.286442  0.207122
minutes   0.982083  1.000000  0.173110  0.280967  0.206955
messages  0.177385  0.173110  1.000000  0.195721  0.203830
mb_used   0.286442  0.280967  0.195721  1.000000  0.198568
is_ultra  0.207122  0.206955  0.203830  0.198568  1.000000


Два признака по сути одно и тоже это calls и minutes. Удалим столбец с кол-вом звонков

In [5]:
df.drop(['calls'], axis=1 , inplace = True)

##  Разбейте данные на выборки

Так как, по задаче нам нужно сделать предложение клиентам наиболее подходящее им,  исходя из  их расхода услуг , то "is_ultra" - это и есть целевой признак.  

In [6]:
X = df.drop(["is_ultra"], axis=1)
Y = df["is_ultra"]

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую

Сет данных небольшой. В этом случае воспользуемся пропорцией 80:20 (обучающая/валидационная)

In [7]:
X_train, X_valid,Y_train, Y_valid  = train_test_split(X,Y,  test_size=0.20, random_state=12345)

Затем валидационную выборку делим еще раз на тестовую и валидационную. 

In [8]:
X_val, X_test, Y_val, Y_test  = train_test_split(X_valid,Y_valid,  test_size=0.50, random_state=12345)

In [9]:
X_train.shape, X_valid.shape, X_val.shape, X_test.shape

((2571, 3), (643, 3), (321, 3), (322, 3))

##  Исследуйте модели

###  Решающее дерево 

In [10]:
model_DTC = DecisionTreeClassifier()
model_DTC.fit(X_train, Y_train)
predict = model_DTC.predict(X_val)
score = model_DTC.score(X_val,Y_val)
train_score =  model_DTC.score(X_train,Y_train)
print('accuracy_valid:', score)
print('accuracy_train:', train_score)

accuracy_valid: 0.7538940809968847
accuracy_train: 1.0


In [11]:
print(classification_report(Y_val, predict))

              precision    recall  f1-score   support

           0       0.80      0.85      0.82       218
           1       0.64      0.54      0.59       103

    accuracy                           0.75       321
   macro avg       0.72      0.70      0.71       321
weighted avg       0.75      0.75      0.75       321



Решающее дерево дает accuracy 0.74 на валидной выборке и 1 на тренировочной. Модель переобучена. 
Точность решения для класса 0 - 0.81  для 1 - только 0.61 (очень близко к рандомному предсказанию)

Подберем гиперпараметры (глубину дерева)

In [12]:
best_score = 0 
for depth in range(2, 11):
    model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_DTC.fit(X_train, Y_train)
    score = model_DTC.score(X_val,Y_val)
    if  score >  best_score : 
        best_score = score
        max_depth = depth

print('accuracy:', score) #Комментарий наставника: лучше выводи здесь лучшую оценку, а не последнюю
print("max_depth =", max_depth)


accuracy: 0.7881619937694704
max_depth = 4


Посмотрим как модель работает с классами 

In [13]:
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=6)
model_DTC.fit(X_train, Y_train)
predict = model_DTC.predict(X_val)
score = model_DTC.score(X_val,Y_val)
train_score =  model_DTC.score(X_train,Y_train)
print('accuracy_valid:', score)
print('accuracy_train:', train_score)
print(classification_report(Y_val, predict))

accuracy_valid: 0.7850467289719626
accuracy_train: 0.8171917541812524
              precision    recall  f1-score   support

           0       0.78      0.95      0.86       218
           1       0.81      0.43      0.56       103

    accuracy                           0.79       321
   macro avg       0.80      0.69      0.71       321
weighted avg       0.79      0.79      0.76       321



На валидационной выборке лучшее accuracy = 0,79 при max_depth = 6. Точность предсказания классов для для 0 - 0.8 . Для 1 - 0.79

### Случайный лес

Зададим параметры  кол-во деревьев 100, глубину выбирем по "accuracy"

In [14]:
best_score = 0 
for depth in range(2, 11):
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=depth)
    model_RFC.fit(X_train, Y_train)
    score = model_RFC.score(X_val,Y_val)
    if  score >  best_score : 
        best_score = score
        max_depth = depth
print('accuracy:', score) #Комментарий наставника: и здесь тоже выводи лучшую оценку, а не последнюю
print("max_depth =", max_depth)


accuracy: 0.8006230529595015
max_depth = 7


In [15]:
model_RFC = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=8)
model_RFC.fit(X_train, Y_train)
predict = model_RFC.predict(X_val)
score = model_RFC.score(X_val,Y_val)
train_score =  model_RFC.score(X_train,Y_train)
print('accuracy_valid:', score)
print('accuracy_train:', train_score)
print(classification_report(Y_val, predict))

accuracy_valid: 0.8068535825545171
accuracy_train: 0.8661999222092571
              precision    recall  f1-score   support

           0       0.81      0.94      0.87       218
           1       0.81      0.52      0.64       103

    accuracy                           0.81       321
   macro avg       0.81      0.73      0.75       321
weighted avg       0.81      0.81      0.79       321



В данной модели accuracy выше - 0.8. Точность по классам примерно такая же как для дерева. 

###  Логистическая регрессия 

In [16]:
model_LR  = LogisticRegression()
model_LR.fit(X_train, Y_train)
predict = model_LR.predict(X_val)
score = model_LR.score(X_val,Y_val)
train_score =  model_LR.score(X_train,Y_train)
print('accuracy_valid:', score)
print('accuracy_train:', train_score)
print(classification_report(Y_val, predict))

accuracy_valid: 0.6915887850467289
accuracy_train: 0.705173084402956
              precision    recall  f1-score   support

           0       0.69      1.00      0.81       218
           1       1.00      0.04      0.07       103

    accuracy                           0.69       321
   macro avg       0.84      0.52      0.44       321
weighted avg       0.79      0.69      0.58       321



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


В данной модели показатели ниже , accuracy - 0.69. 

## Проверьте модель на тестовой выборке

Проверим модель на тестовой выборке по модели случайного леса

In [17]:
predict = model_RFC.predict(X_test)
score = model_RFC.score(X_test,Y_test)
print('accuracy_test:', score)
print(classification_report(Y_test, predict))

accuracy_test: 0.7981366459627329
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       229
           1       0.73      0.47      0.58        93

    accuracy                           0.80       322
   macro avg       0.77      0.70      0.72       322
weighted avg       0.79      0.80      0.78       322



То же проделаем для модели решающего дерева 

In [18]:
predict = model_DTC.predict(X_test)
score = model_DTC.score(X_test,Y_test)
print('accuracy_test:', score)
print(classification_report(Y_test, predict))

accuracy_test: 0.7763975155279503
              precision    recall  f1-score   support

           0       0.78      0.96      0.86       229
           1       0.76      0.33      0.46        93

    accuracy                           0.78       322
   macro avg       0.77      0.64      0.66       322
weighted avg       0.77      0.78      0.74       322



В качестве модели выбираем модель случайного леса. Она лучше проявила себя на тестовой выборке. А так же сильно лучше работает
с различными классами

## Проверьте модели на адекватность

In [19]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Вывод: 

Наиболее точное решение дает модель с алгоритмом "Случайный лес". Она немногим более точная чем алгоритм "Дерево решений"

Если время расчета критично, можно вомпользоваться менее затратным алгоритмом.